## Speed for three Trajectory classes in `pytraj`

What are they?
    * TrajReadOnly: a thin wrapper for cpptraj Trajin classes. Data will only be loaded if iterating it
    * FrameArray: a wrapper of C++ vector of Frame object. Data will be fully loaded to memory. This class was originally designed to hold in-memory Frames for TrajReadOnly and for faster iterating
    * api.Trajectory: new class using numpy
        * why this class: it's very fast to iterate FrameArray but it's a bit slow to convert to numpy array. I created this class as an interface to numpy/scipy/mdtraj ....
        * Note: FrameArray might be deprecated, depending on how I (we) really need. For technical diffculty in manage lifetime of C++ vector in Python, I can not subclass FrameArray in Python level. 

In [1]:
from pytraj.__version__ import __version__ as p_version
import mdtraj as md
print (p_version)

0.1.2.dev3


## loading data

In [2]:
# data is stored in local disk, generated from TIP3P REMD, netcdf, 200 Mb, 17443 atoms, 1000 frames

from pytraj import io
import mdtraj as md

top_name = "../tests/data/nogit/remd/myparm.parm7"
filename = "../tests/data/nogit/remd/remd.x.000"
file_and_top_names = (filename, top_name)

In [3]:
# load classes
from pytraj import FrameArray
from pytraj.api import Trajectory
from pytraj import TrajReadOnly

In [4]:
%time fa = FrameArray(*file_and_top_names)
%time traj = Trajectory(*file_and_top_names)
%time trajread = TrajReadOnly(*file_and_top_names)

CPU times: user 2.39 s, sys: 869 ms, total: 3.26 s
Wall time: 4.15 s
CPU times: user 2.27 s, sys: 750 ms, total: 3.02 s
Wall time: 3.03 s
CPU times: user 204 ms, sys: 16 ms, total: 220 ms
Wall time: 220 ms


In [5]:
# reload since we used "%timeit".

fa = FrameArray(*file_and_top_names)
traj = Trajectory(*file_and_top_names)
trajread = TrajReadOnly(*file_and_top_names)

## iterate frames

In [6]:
%timeit for frame in fa: pass
%timeit for frame in traj: pass
%timeit for frame in trajread: pass

# iterate FrameArray is extremely fast since it is a wrapper of C++ vector class

1000 loops, best of 3: 822 µs per loop
1 loops, best of 3: 2.25 s per loop
1 loops, best of 3: 1.02 s per loop


## frame_iter

In [7]:
start, stop, stride = 10, 999, 100

%timeit for frame in fa.frame_iter(start, stop, stride): pass
%timeit for frame in traj.frame_iter(start, stop, stride): pass
%timeit for frame in trajread.frame_iter(start, stop, stride): pass

# we need to redesign frame_iter of Trajectory

1000 loops, best of 3: 1.64 ms per loop
10 loops, best of 3: 39.6 ms per loop
100 loops, best of 3: 19 ms per loop


## convert to numpy array by calling `xyz` attribute

In [8]:
%timeit fa.xyz
%timeit traj.xyz
%timeit trajread.xyz

# traj.xyz is numpy array, so it tooks only 115 ns to call (vs. 504 ms for FrameArray and 2.6 s for TrajReadOnly).
# However iterating Trajectory (numpy) is much slower.
# the best thing is to move numpy to Cython level so we can combine fast iterating in Cython + fast vectorizing in numpy

1 loops, best of 3: 504 ms per loop
The slowest run took 26.04 times longer than the fastest. This could mean that an intermediate result is being cached 
10000000 loops, best of 3: 115 ns per loop
1 loops, best of 3: 2.6 s per loop


## calc_rmsd

In [9]:
# save reference
f0 = fa[0]
%timeit for frame in fa: frame.rmsd(f0) 
%timeit for frame in traj: frame.rmsd(f0) 
%timeit for frame in trajread: frame.rmsd(f0)

# we can use fast rmsd calculation in mdtraj for our Trajectory too. (make the rmsd calculation ~ 3 times faster)
# for this calculation, mdtraj only cares about `xyz` attribute, and all of Trajectories in pytraj do have `xyz`
%timeit md.rmsd(traj, traj, 0)

1 loops, best of 3: 978 ms per loop
1 loops, best of 3: 3.25 s per loop
1 loops, best of 3: 1.91 s per loop
1 loops, best of 3: 391 ms per loop


## calc_radgyr

In [10]:
%timeit fa.calc_radgyr()
%timeit traj.calc_radgyr()
%timeit trajread.calc_radgyr()

# trajread is the slowest because it needs to load frames to memory

1 loops, best of 3: 288 ms per loop
1 loops, best of 3: 2.87 s per loop
1 loops, best of 3: 1.24 s per loop


## calc_dssp

In [11]:
%timeit fa.calc_dssp()
%timeit traj.calc_dssp()
%timeit trajread.calc_dssp()

10 loops, best of 3: 167 ms per loop
1 loops, best of 3: 2.91 s per loop
1 loops, best of 3: 1.2 s per loop


## calc_COM

In [12]:
%timeit fa.calc_COM()
%timeit traj.calc_COM()
%timeit trajread.calc_COM()

1 loops, best of 3: 379 ms per loop
1 loops, best of 3: 4.58 s per loop
1 loops, best of 3: 1.23 s per loop
